In [34]:
import numpy as np
import pandas as pd

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os

In [36]:
data_dir = (r'E:\Running_projects\Human Activity Recognition through Ensemble Learning of Multiple CNN Mod 2\M2\testing_set')
categories = ['ApplyEyeMakeup','ApplyLipstick','Archery']

In [37]:
for i in categories:
    path = os.path.join(data_dir, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))    

In [38]:
img_size = 128
image_array = cv2.resize(img_array, (img_size,img_size))

In [39]:
train_data = []

for i in categories:
    train_path = os.path.join(data_dir,i)
    tag = categories.index(i)
    for img in os.listdir(train_path):
        try:
            image_arr = cv2.imread(os.path.join(train_path , img), cv2.IMREAD_GRAYSCALE)
            new_image_array = cv2.resize(image_arr, (img_size,img_size))
            train_data.append([new_image_array , tag])
        except Exception as e:
            pass

In [40]:
X = []
y = []
for i,j in train_data:
    X.append(i)
    y.append(j)
X = np.array(X).reshape(-1,img_size,img_size)
print(X.shape)
X = X/255.0  
X = X.reshape(-1,128,128,1)

(450, 128, 128)


In [41]:
from keras.utils.np_utils import to_categorical   
y_enc = to_categorical(y, num_classes = 24)

In [42]:
X_train , X_test, y_train, y_test = train_test_split(X , y_enc , test_size = 0.1, random_state = 42)
X_train , X_val, y_train, y_val = train_test_split(X_train , y_train , test_size = 0.1, random_state = 42)

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [44]:
import itertools
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

In [45]:
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
#from keras.optimizers import RMSprop,Adam (if you use 2019 anaconda , please uncomment this line(remove "#") )
from tensorflow.keras.optimizers import RMSprop,Adam #(This line For 2020 anaconda. But If you use 2019 , comment this line with "#" )

In [46]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (128,128,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(24, activation = "softmax"))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])

epochs = 1

es = EarlyStopping(
    monitor='val_acc', 
    mode='max',
    patience = 3
)

batch_size = 16
imggen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=0,
        zoom_range = 0,
        width_shift_range=0,  
        height_shift_range=0,  
        horizontal_flip=True,  
        vertical_flip=False)


C:\Users\gts\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [47]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("models/model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

imggen.fit(X_train)
history = model.fit_generator(imggen.flow(X_train,y_train,batch_size = batch_size),
                              epochs = epochs, validation_data = (X_val,y_val),
                              steps_per_epoch = X_train.shape[0] // batch_size,
                              callbacks=callbacks_list)

C:\Users\gts\AppData\Local\Temp/ipykernel_10192/19206160.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(imggen.flow(X_train,y_train,batch_size = batch_size),


22/22 [==============================] - 48s 2s/step - loss: 0.1779 - accuracy: 0.3190 - val_loss: 0.1070 - val_accuracy: 0.2927


In [56]:
model_json = model.to_json()
with open(r".\models\resnet-34_kinetics.onnx") as json_file:
    json_file.write(model_json)

FileNotFoundError: [Errno 2] No such file or directory: '.\\models\\resnet-34_kinetics.onnx'

In [55]:
os.listdir(r"models")

['human.data-00000-of-00001']